# AI Config Context (Advanced) - Cookbook

Tests all code patterns from `aiconfig-context-advanced/SKILL.md`.

Prerequisites: `LAUNCHDARKLY_SDK_KEY`

In [1]:
%pip install launchdarkly-server-sdk python-dotenv -q

import os
from pathlib import Path
from dotenv import load_dotenv

def find_repo_root(start_path: Path = None) -> Path:
    current = start_path or Path.cwd()
    for parent in [current] + list(current.parents):
        if (parent / '.git').exists():
            return parent
    return current

repo_root = find_repo_root()
load_dotenv(repo_root / '.env')
print(f"[OK] Loaded environment from {repo_root / '.env'}")


[notice] A new release of pip is available: 25.1.1 -> 26.0
[notice] To update, run: /opt/homebrew/opt/python@3.11/bin/python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
[OK] Loaded environment from /Users/ld_scarlett/Documents/Github/agent-skills/.env


In [2]:
# Common imports for all tests
from ldclient import Context
import hashlib
from datetime import datetime
import time
import json

print("[OK] Imports ready")

[OK] Imports ready


---
## 1. Experimentation & Testing Context
From: `SKILL.md` lines 36-74

In [3]:
def calculate_age_days(created_at):
    """Helper: Calculate account age in days"""
    if not created_at:
        return 0
    if isinstance(created_at, str):
        created_at = datetime.fromisoformat(created_at)
    return (datetime.now() - created_at).days

def categorize_usage(api_calls: int) -> str:
    """Categorize usage levels for segmentation"""
    if api_calls > 100000:
        return "power_user"
    elif api_calls > 10000:
        return "active"
    elif api_calls > 1000:
        return "regular"
    else:
        return "light"

def build_experiment_context(user_id: str, user_data: dict):
    """Build context with experiment-relevant attributes"""

    builder = Context.builder(user_id)

    # Core segmentation attributes
    builder.set("tier", user_data.get("subscription_tier", "free"))
    builder.set("accountAgeDays", calculate_age_days(user_data.get("created_at")))
    builder.set("usageLevel", categorize_usage(user_data.get("api_calls_monthly", 0)))

    # Experiment cohort assignment
    builder.set("experimentCohort", user_data.get("cohort", "default"))

    # Stable bucketing for consistent assignment
    bucket = int(hashlib.md5(user_id.encode()).hexdigest()[:8], 16) % 100
    builder.set("experimentBucket", bucket)

    # Time-based cohorts for rollout analysis
    week_number = datetime.now().isocalendar()[1]
    builder.set("cohortWeek", week_number)

    return builder.build()

print("[OK] build_experiment_context() defined")

[OK] build_experiment_context() defined


In [4]:
# Test build_experiment_context
print("=== Testing build_experiment_context ===")
ctx = build_experiment_context("user-123", {
    "subscription_tier": "premium",
    "api_calls_monthly": 50000,
    "cohort": "beta"
})
print(f"[OK] Context key: {ctx.key}")
print(f"     tier: {ctx.get('tier')}")
print(f"     usageLevel: {ctx.get('usageLevel')}")
print(f"     experimentBucket: {ctx.get('experimentBucket')}")

# Test categorize_usage
print("\n=== Testing categorize_usage ===")
assert categorize_usage(500) == "light"
assert categorize_usage(5000) == "regular"
assert categorize_usage(50000) == "active"
assert categorize_usage(500000) == "power_user"
print("[OK] categorize_usage() tests pass")

=== Testing build_experiment_context ===
[OK] Context key: user-123
     tier: premium
     usageLevel: active
     experimentBucket: 16

=== Testing categorize_usage ===
[OK] categorize_usage() tests pass


---
## 2. Targeted Deployments & Canary Testing
From: `SKILL.md` lines 90-126

In [5]:
def map_to_datacenter(region):
    """Helper: Map region to datacenter"""
    mapping = {
        "us-east": "us-east-1",
        "us-west": "us-west-2",
        "eu-west": "eu-west-1",
        "ap-southeast": "ap-southeast-1"
    }
    return mapping.get(region, "us-east-1")

def calculate_performance_tier(user_data: dict) -> str:
    """Determine performance tier for capacity planning"""
    if user_data.get("plan") == "enterprise" and user_data.get("sla") == "premium":
        return "high_priority"
    elif user_data.get("plan") in ["business", "enterprise"]:
        return "standard"
    else:
        return "best_effort"

def build_deployment_context(user_id: str, user_data: dict, org_data: dict = None):
    """Context for targeted deployments and canary testing"""

    builder = Context.builder(user_id)

    # Risk tolerance indicators
    builder.set("betaTester", user_data.get("opted_into_beta", False))

    # Parse date properly for comparison
    signup_date = datetime.fromisoformat(user_data.get("signup_date", "2024-01-01"))
    builder.set("earlyAdopter", signup_date < datetime(2023, 1, 1))

    # Organizational risk profile
    if org_data:
        builder.set("orgRiskTolerance", org_data.get("risk_profile", "conservative"))
        builder.set("orgSize", org_data.get("employee_count", 0))

    # Geographic targeting
    builder.set("region", user_data.get("region", "us-east"))
    builder.set("dataCenter", map_to_datacenter(user_data.get("region")))

    # Performance tier for capacity management
    builder.set("performanceTier", calculate_performance_tier(user_data))

    return builder.build()

print("[OK] build_deployment_context() defined")

[OK] build_deployment_context() defined


In [6]:
# Test build_deployment_context
print("=== Testing build_deployment_context ===")
ctx = build_deployment_context(
    "user-456",
    {"opted_into_beta": True, "signup_date": "2022-06-15", "region": "eu-west", "plan": "enterprise", "sla": "premium"},
    {"risk_profile": "aggressive", "employee_count": 500}
)
print(f"[OK] Context key: {ctx.key}")
print(f"     betaTester: {ctx.get('betaTester')}")
print(f"     earlyAdopter: {ctx.get('earlyAdopter')}")
print(f"     dataCenter: {ctx.get('dataCenter')}")
print(f"     performanceTier: {ctx.get('performanceTier')}")

# Test calculate_performance_tier
print("\n=== Testing calculate_performance_tier ===")
assert calculate_performance_tier({"plan": "enterprise", "sla": "premium"}) == "high_priority"
assert calculate_performance_tier({"plan": "business"}) == "standard"
assert calculate_performance_tier({"plan": "free"}) == "best_effort"
print("[OK] calculate_performance_tier() tests pass")

=== Testing build_deployment_context ===
[OK] Context key: user-456
     betaTester: True
     earlyAdopter: True
     dataCenter: eu-west-1
     performanceTier: high_priority

=== Testing calculate_performance_tier ===
[OK] calculate_performance_tier() tests pass


---
## 3. Multi-Context for B2B Applications
From: `SKILL.md` lines 135-178

In [7]:
def categorize_org_size(employee_count: int) -> str:
    """Categorize organization size"""
    if employee_count is None:
        return "startup"
    if employee_count > 1000:
        return "enterprise"
    elif employee_count > 100:
        return "mid_market"
    elif employee_count > 10:
        return "small_business"
    else:
        return "startup"

def build_b2b_contexts(user_data: dict, org_data: dict, workspace_data: dict = None):
    """Build comprehensive B2B multi-context"""

    # User context
    user_context = Context.builder(user_data["id"]).kind("user")
    user_context.set("role", user_data.get("role"))
    user_context.set("department", user_data.get("department"))
    user_context.set("permissions", user_data.get("permission_level"))
    user_context.set("lastActive", user_data.get("last_activity"))

    # Organization context
    org_context = Context.builder(org_data["id"]).kind("organization")
    org_context.set("tier", org_data.get("plan"))
    org_context.set("industry", org_data.get("industry"))
    org_context.set("size", categorize_org_size(org_data.get("employee_count")))
    org_context.set("mrr", org_data.get("monthly_revenue"))
    org_context.set("features", org_data.get("enabled_features", []))

    # Optional workspace/team context
    multi_builder = Context.multi_builder()
    multi_builder.add(user_context.build())
    multi_builder.add(org_context.build())

    if workspace_data:
        workspace_context = Context.builder(workspace_data["id"]).kind("workspace")
        workspace_context.set("name", workspace_data.get("name"))
        workspace_context.set("type", workspace_data.get("type"))
        workspace_context.set("memberCount", workspace_data.get("member_count"))
        multi_builder.add(workspace_context.build())

    return multi_builder.build()

print("[OK] build_b2b_contexts() defined")

[OK] build_b2b_contexts() defined


In [8]:
# Test build_b2b_contexts
print("=== Testing build_b2b_contexts ===")
ctx = build_b2b_contexts(
    {"id": "user-789", "role": "admin", "department": "engineering"},
    {"id": "org-123", "plan": "enterprise", "industry": "fintech", "employee_count": 500},
    {"id": "ws-456", "name": "Backend Team", "type": "engineering", "member_count": 12}
)
print(f"[OK] Multi-context created")
print(f"     Multiple: {ctx.multiple}")

# Test categorize_org_size
print("\n=== Testing categorize_org_size ===")
assert categorize_org_size(5) == "startup"
assert categorize_org_size(50) == "small_business"
assert categorize_org_size(500) == "mid_market"
assert categorize_org_size(5000) == "enterprise"
print("[OK] categorize_org_size() tests pass")

=== Testing build_b2b_contexts ===
[OK] Multi-context created
     Multiple: True

=== Testing categorize_org_size ===
[OK] categorize_org_size() tests pass


---
## 4. Context Enrichment Pipeline
From: `SKILL.md` lines 188-248

In [9]:
# Placeholder helper functions for enrichment
def get_account_value(user_id):
    return 10000

def get_industry_segment(company):
    return "technology"

def get_daily_active_use(user_id):
    return True

def calculate_feature_adoption(user_id):
    return 0.75

def calculate_engagement_score(value, active):
    return 85

class ContextEnrichmentPipeline:
    """Pipeline for enriching contexts with external data"""

    def __init__(self):
        self.enrichers = []

    def add_enricher(self, enricher):
        """Add an enrichment step"""
        self.enrichers.append(enricher)
        return self

    def enrich(self, base_context_data: dict) -> Context:
        """Build enriched context"""
        user_id = base_context_data["user_id"]
        builder = Context.builder(user_id)

        # Apply each enricher
        enriched_data = base_context_data.copy()
        for enricher in self.enrichers:
            enriched_data = enricher(enriched_data)

        # Build context with enriched data
        for key, value in enriched_data.items():
            if key != "user_id" and value is not None:
                builder.set(key, value)

        return builder.build()

# Example enrichers
def enrich_from_crm(data: dict) -> dict:
    """Enrich with CRM data"""
    data["accountValue"] = get_account_value(data.get("user_id"))
    data["industrySegment"] = get_industry_segment(data.get("company"))
    return data

def enrich_from_usage_metrics(data: dict) -> dict:
    """Enrich with usage analytics"""
    data["dailyActiveUse"] = get_daily_active_use(data.get("user_id"))
    data["featureAdoption"] = calculate_feature_adoption(data.get("user_id"))
    return data

def enrich_with_computed_fields(data: dict) -> dict:
    """Add computed attributes"""
    if data.get("accountValue") and data.get("dailyActiveUse"):
        data["engagementScore"] = calculate_engagement_score(
            data["accountValue"],
            data["dailyActiveUse"]
        )
    return data

print("[OK] ContextEnrichmentPipeline defined")

[OK] ContextEnrichmentPipeline defined


In [10]:
# Test ContextEnrichmentPipeline
print("=== Testing ContextEnrichmentPipeline ===")
pipeline = ContextEnrichmentPipeline()
pipeline.add_enricher(enrich_from_crm) \
        .add_enricher(enrich_from_usage_metrics) \
        .add_enricher(enrich_with_computed_fields)

ctx = pipeline.enrich({"user_id": "user-123", "company": "TechCorp"})
print(f"[OK] Context key: {ctx.key}")
print(f"     accountValue: {ctx.get('accountValue')}")
print(f"     industrySegment: {ctx.get('industrySegment')}")
print(f"     engagementScore: {ctx.get('engagementScore')}")

=== Testing ContextEnrichmentPipeline ===
[OK] Context key: user-123
     accountValue: 10000
     industrySegment: technology
     engagementScore: 85


---
## 5. JWT Claims Integration
From: `SKILL.md` lines 254-305

In [11]:
def context_from_jwt_with_custom_claims(jwt_claims: dict, claim_mappings: dict = None):
    """Build context from JWT with custom claim mappings"""

    # Standard claims
    user_id = jwt_claims.get("sub") or jwt_claims.get("user_id")
    if not user_id:
        raise ValueError("No user identifier in JWT")

    builder = Context.builder(user_id)

    # Default claim mappings
    default_mappings = {
        "email": "email",
        "name": "name",
        "given_name": "firstName",
        "family_name": "lastName",
        "preferred_username": "username",
        "locale": "locale",
        "zoneinfo": "timezone",
        # Custom claims
        "org_id": "organizationId",
        "team_id": "teamId",
        "subscription_tier": "tier",
        "permissions": "permissions"
    }

    # Merge with provided mappings
    if claim_mappings:
        default_mappings.update(claim_mappings)

    # Map claims to context
    for claim_key, context_key in default_mappings.items():
        if claim_key in jwt_claims:
            builder.set(context_key, jwt_claims[claim_key])

    # Handle nested custom claims
    if "custom" in jwt_claims:
        for key, value in jwt_claims["custom"].items():
            # Flatten nested structures
            if isinstance(value, dict):
                for nested_key, nested_value in value.items():
                    builder.set(f"{key}_{nested_key}", nested_value)
            else:
                builder.set(key, value)

    # Add JWT metadata
    builder.set("jwtIssuer", jwt_claims.get("iss"))
    builder.set("jwtAudience", jwt_claims.get("aud"))

    return builder.build()

print("[OK] context_from_jwt_with_custom_claims() defined")

[OK] context_from_jwt_with_custom_claims() defined


In [12]:
# Test context_from_jwt_with_custom_claims
print("=== Testing context_from_jwt_with_custom_claims ===")
jwt_claims = {
    "sub": "user-jwt-123",
    "email": "user@example.com",
    "name": "John Doe",
    "given_name": "John",
    "org_id": "org-456",
    "subscription_tier": "premium",
    "iss": "https://auth.example.com",
    "aud": "my-app",
    "custom": {
        "feature_flags": {"beta": True, "v2": False},
        "role": "admin"
    }
}
ctx = context_from_jwt_with_custom_claims(jwt_claims)
print(f"[OK] Context key: {ctx.key}")
print(f"     email: {ctx.get('email')}")
print(f"     firstName: {ctx.get('firstName')}")
print(f"     organizationId: {ctx.get('organizationId')}")
print(f"     tier: {ctx.get('tier')}")
print(f"     feature_flags_beta: {ctx.get('feature_flags_beta')}")
print(f"     role: {ctx.get('role')}")

=== Testing context_from_jwt_with_custom_claims ===
[OK] Context key: user-jwt-123
     email: user@example.com
     firstName: John
     organizationId: org-456
     tier: premium
     feature_flags_beta: True
     role: admin


---
## 6. Cardinality Safety: Bucket Numeric Fields
From: `SKILL.md` lines 437-445

In [13]:
def bucket_revenue(revenue: float) -> str:
    if revenue < 1000: return "0-1k"
    elif revenue < 10000: return "1k-10k"
    elif revenue < 100000: return "10k-100k"
    else: return "100k+"

print("[OK] bucket_revenue() defined")

[OK] bucket_revenue() defined


In [14]:
# Test bucket_revenue
print("=== Testing bucket_revenue ===")
assert bucket_revenue(500) == "0-1k"
assert bucket_revenue(5000) == "1k-10k"
assert bucket_revenue(50000) == "10k-100k"
assert bucket_revenue(500000) == "100k+"
print("[OK] bucket_revenue() tests pass")

# Example usage in context
builder = Context.builder("user-123")
builder.set("revenueRange", bucket_revenue(15234.56))
ctx = builder.build()
print(f"[OK] revenueRange: {ctx.get('revenueRange')}")

=== Testing bucket_revenue ===
[OK] bucket_revenue() tests pass
[OK] revenueRange: 10k-100k


---
## 7. Geographic Context Patterns
From: `SKILL.md` lines 478-521

In [15]:
# Helper functions for geographic context
def map_to_continent(country_code):
    mapping = {"US": "NA", "CA": "NA", "GB": "EU", "DE": "EU", "FR": "EU", "JP": "AS", "AU": "OC"}
    return mapping.get(country_code, "OTHER")

def map_to_market_region(country_code):
    if country_code in ["US", "CA"]:
        return "north_america"
    elif country_code in ["GB", "DE", "FR", "IT", "ES"]:
        return "emea"
    elif country_code in ["JP", "KR", "AU", "SG"]:
        return "apac"
    return "other"

def infer_language(country_code):
    mapping = {"US": "en", "GB": "en", "DE": "de", "FR": "fr", "JP": "ja", "ES": "es"}
    return mapping.get(country_code, "en")

def infer_currency(country_code):
    mapping = {"US": "USD", "GB": "GBP", "DE": "EUR", "FR": "EUR", "JP": "JPY"}
    return mapping.get(country_code, "USD")

def is_gdpr_country(country_code):
    eu_countries = ["DE", "FR", "IT", "ES", "NL", "BE", "PL", "SE", "DK", "FI", "AT", "IE", "PT"]
    return country_code in eu_countries

def get_privacy_region(country_code: str) -> str:
    """Determine data privacy region"""
    eu_countries = ["DE", "FR", "IT", "ES", "NL", "BE", "PL", "SE", "DK", "FI"]
    if country_code in eu_countries:
        return "eu"
    elif country_code == "US":
        return "us"
    elif country_code in ["JP", "KR"]:
        return "apac_strict"
    else:
        return "standard"

def build_geographic_context_advanced(user_id: str, ip_data: dict, preferences: dict = None):
    """Build context with advanced geographic attributes"""

    builder = Context.builder(user_id)

    # Basic geographic data
    builder.set("country", ip_data.get("country_code"))
    builder.set("region", ip_data.get("region"))
    builder.set("city", ip_data.get("city"))
    builder.set("timezone", ip_data.get("timezone"))

    # Advanced geographic attributes
    builder.set("continent", map_to_continent(ip_data.get("country_code")))
    builder.set("marketRegion", map_to_market_region(ip_data.get("country_code")))
    builder.set("dataPrivacyRegion", get_privacy_region(ip_data.get("country_code")))

    # Language and localization
    if preferences:
        builder.set("preferredLanguage", preferences.get("language"))
        builder.set("preferredCurrency", preferences.get("currency"))
    else:
        # Infer from location
        builder.set("inferredLanguage", infer_language(ip_data.get("country_code")))
        builder.set("inferredCurrency", infer_currency(ip_data.get("country_code")))

    # Compliance and regulations
    builder.set("gdprApplies", is_gdpr_country(ip_data.get("country_code")))
    builder.set("ccpaApplies", ip_data.get("region") == "CA" and ip_data.get("country_code") == "US")

    return builder.build()

print("[OK] build_geographic_context_advanced() defined")

[OK] build_geographic_context_advanced() defined


In [16]:
# Test build_geographic_context_advanced
print("=== Testing build_geographic_context_advanced ===")
ctx = build_geographic_context_advanced(
    "user-geo-123",
    {"country_code": "DE", "region": "Bavaria", "city": "Munich", "timezone": "Europe/Berlin"}
)
print(f"[OK] Context key: {ctx.key}")
print(f"     country: {ctx.get('country')}")
print(f"     continent: {ctx.get('continent')}")
print(f"     marketRegion: {ctx.get('marketRegion')}")
print(f"     dataPrivacyRegion: {ctx.get('dataPrivacyRegion')}")
print(f"     gdprApplies: {ctx.get('gdprApplies')}")
print(f"     inferredLanguage: {ctx.get('inferredLanguage')}")

# Test get_privacy_region
print("\n=== Testing get_privacy_region ===")
assert get_privacy_region("DE") == "eu"
assert get_privacy_region("US") == "us"
assert get_privacy_region("JP") == "apac_strict"
assert get_privacy_region("BR") == "standard"
print("[OK] get_privacy_region() tests pass")

=== Testing build_geographic_context_advanced ===
[OK] Context key: user-geo-123
     country: DE
     continent: EU
     marketRegion: emea
     dataPrivacyRegion: eu
     gdprApplies: True
     inferredLanguage: de

=== Testing get_privacy_region ===
[OK] get_privacy_region() tests pass


---
## 8. Agent Graph Context Patterns
From: `SKILL.md` lines 527-563

In [17]:
def build_agent_graph_context(
    graph_id: str,
    node_id: str,
    graph_data: dict,
    node_data: dict
) -> dict:
    """Build context for agent graph execution"""

    # Graph-level context
    graph_context = Context.builder(graph_id).kind("agentGraph")
    graph_context.set("graphVersion", graph_data.get("version", "1.0"))
    graph_context.set("graphStage", graph_data.get("stage", "prod"))  # draft/test/prod
    graph_context.set("graphOwnerTeam", graph_data.get("owner_team"))
    graph_context.set("graphPolicyProfile", graph_data.get("policy", "standard"))
    graph_context.set("graphDomain", graph_data.get("domain"))  # claims/underwriting/etc

    # Node-level context
    node_context = Context.builder(node_id).kind("agentNode")
    node_context.set("nodeType", node_data.get("type"))  # planner/retriever/executor
    node_context.set("nodeDepth", node_data.get("depth", 0))
    node_context.set("parentNodeId", node_data.get("parent_id"))
    node_context.set("nodeCriticality", node_data.get("criticality", "medium"))

    return {
        "graph": graph_context.build(),
        "node": node_context.build()
    }

print("[OK] build_agent_graph_context() defined")

[OK] build_agent_graph_context() defined


In [18]:
# Test build_agent_graph_context
print("=== Testing build_agent_graph_context ===")
contexts = build_agent_graph_context(
    graph_id="claims-triage-v3",
    node_id="node-retriever-01",
    graph_data={"version": "3.2", "stage": "canary", "owner_team": "claims", "domain": "claims"},
    node_data={"type": "retriever", "depth": 1, "criticality": "high"}
)
print(f"[OK] Graph context key: {contexts['graph'].key}")
print(f"     graphVersion: {contexts['graph'].get('graphVersion')}")
print(f"     graphStage: {contexts['graph'].get('graphStage')}")
print(f"[OK] Node context key: {contexts['node'].key}")
print(f"     nodeType: {contexts['node'].get('nodeType')}")
print(f"     nodeCriticality: {contexts['node'].get('nodeCriticality')}")

=== Testing build_agent_graph_context ===
[OK] Graph context key: claims-triage-v3
     graphVersion: 3.2
     graphStage: canary
[OK] Node context key: node-retriever-01
     nodeType: retriever
     nodeCriticality: high


---
## 9. Orchestrator & Runtime Context
From: `SKILL.md` lines 569-596

In [19]:
def build_orchestrator_context(
    orchestrator_type: str,
    runtime_data: dict
) -> Context:
    """Build context for orchestration runtime"""

    # Use stable orchestrator ID
    orchestrator_id = f"{orchestrator_type}-{runtime_data.get('region', 'default')}"
    builder = Context.builder(orchestrator_id).kind("orchestrator")

    # Core orchestrator attributes
    builder.set("orchestratorType", orchestrator_type)  # langgraph/temporal/custom
    builder.set("orchestratorVersion", runtime_data.get("version"))
    builder.set("executionMode", runtime_data.get("mode", "async"))  # sync/async/streaming

    # Runtime configuration
    builder.set("retryPolicy", runtime_data.get("retry_policy", "standard"))
    builder.set("parallelismEnabled", runtime_data.get("parallel", False))
    builder.set("checkpointing", runtime_data.get("checkpointing", True))
    builder.set("stateBackend", runtime_data.get("state_backend", "memory"))

    # Performance characteristics
    builder.set("timeoutBudgetMs", runtime_data.get("timeout_ms", 30000))
    builder.set("maxRetries", runtime_data.get("max_retries", 3))

    return builder.build()

print("[OK] build_orchestrator_context() defined")

[OK] build_orchestrator_context() defined


In [20]:
# Test build_orchestrator_context
print("=== Testing build_orchestrator_context ===")
ctx = build_orchestrator_context(
    "langgraph",
    {"region": "us-west", "version": "0.1.5", "mode": "streaming", "parallel": True}
)
print(f"[OK] Context key: {ctx.key}")
print(f"     orchestratorType: {ctx.get('orchestratorType')}")
print(f"     orchestratorVersion: {ctx.get('orchestratorVersion')}")
print(f"     executionMode: {ctx.get('executionMode')}")
print(f"     parallelismEnabled: {ctx.get('parallelismEnabled')}")

=== Testing build_orchestrator_context ===
[OK] Context key: langgraph-us-west
     orchestratorType: langgraph
     orchestratorVersion: 0.1.5
     executionMode: streaming
     parallelismEnabled: True


---
## 10. MCP & Toolchain Context
From: `SKILL.md` lines 609-671

In [21]:
def categorize_count(count: int) -> str:
    """Bucket counts to avoid high cardinality"""
    if count == 0:
        return "none"
    elif count <= 2:
        return "1-2"
    elif count <= 5:
        return "3-5"
    else:
        return "6+"

def risk_priority(risk: str) -> int:
    """Convert risk tier to priority for comparison"""
    return {"low": 1, "medium": 2, "high": 3, "critical": 4}.get(risk, 0)

def build_mcp_context(
    toolchain_profile: str,  # Use predefined profile as key
    mcp_servers: list,
    tool_policy: str = "standard"
) -> Context:
    """
    Build context for MCP servers and toolchain.

    IMPORTANT: Use a predefined profile as key (not a hash of servers)
    to maintain low cardinality. Store server details as private attributes.
    """

    # Use predefined profile as key (e.g., "claims-basic", "underwriting-advanced")
    builder = Context.builder(toolchain_profile).kind("toolchain")

    # Summary attributes (low cardinality, good for targeting)
    builder.set("mcpEnabled", len(mcp_servers) > 0)
    builder.set("mcpServerCount", categorize_count(len(mcp_servers)))
    builder.set("toolPolicy", tool_policy)

    # Derive capability summary from all servers
    max_risk = "low"
    write_capable = False
    third_party_present = False
    server_ids = []

    for server in mcp_servers:
        server_ids.append(server.get("id"))
        risk = server.get("riskTier", "low")
        if risk_priority(risk) > risk_priority(max_risk):
            max_risk = risk

        if server.get("permissions", "read") in ["write", "admin"]:
            write_capable = True

        if server.get("vendor") not in ["internal", "first_party"]:
            third_party_present = True

    builder.set("maxRiskTier", max_risk)
    builder.set("writeCapable", write_capable)
    builder.set("thirdPartyPresent", third_party_present)

    # Optional: Include server list as private attribute for debugging
    builder.set("mcpServerList", server_ids)
    builder.private("mcpServerList")

    return builder.build()

print("[OK] build_mcp_context() defined")

[OK] build_mcp_context() defined


In [22]:
# Test build_mcp_context
print("=== Testing build_mcp_context ===")
ctx = build_mcp_context(
    "claims-standard",
    [
        {"id": "mcp-claims-db", "vendor": "internal", "riskTier": "medium", "permissions": "read"},
        {"id": "mcp-docs", "vendor": "internal", "riskTier": "low", "permissions": "read"},
        {"id": "mcp-external-api", "vendor": "third_party", "riskTier": "high", "permissions": "write"}
    ],
    "strict"
)
print(f"[OK] Context key: {ctx.key}")
print(f"     mcpEnabled: {ctx.get('mcpEnabled')}")
print(f"     mcpServerCount: {ctx.get('mcpServerCount')}")
print(f"     maxRiskTier: {ctx.get('maxRiskTier')}")
print(f"     writeCapable: {ctx.get('writeCapable')}")
print(f"     thirdPartyPresent: {ctx.get('thirdPartyPresent')}")

# Test helper functions
print("\n=== Testing helpers ===")
assert categorize_count(0) == "none"
assert categorize_count(2) == "1-2"
assert categorize_count(4) == "3-5"
assert categorize_count(10) == "6+"
print("[OK] categorize_count() tests pass")

assert risk_priority("low") < risk_priority("medium") < risk_priority("high") < risk_priority("critical")
print("[OK] risk_priority() tests pass")

=== Testing build_mcp_context ===
[OK] Context key: claims-standard
     mcpEnabled: True
     mcpServerCount: 3-5
     maxRiskTier: high
     writeCapable: True
     thirdPartyPresent: True

=== Testing helpers ===
[OK] categorize_count() tests pass
[OK] risk_priority() tests pass


---
## 11. Builder Context (Insurance Employees)
From: `SKILL.md` lines 677-709

In [23]:
def build_builder_context(
    employee_id: str,
    employee_data: dict,
    compliance_data: dict = None
) -> Context:
    """Build context for insurance employee builders"""

    builder = Context.builder(employee_id).kind("builder")

    # Core employee attributes
    builder.set("builderRole", employee_data.get("role"))  # underwriter/claims_adjuster
    builder.set("department", employee_data.get("department"))
    builder.set("seniority", employee_data.get("seniority", "mid"))
    builder.set("trainingStatus", employee_data.get("training", "trained"))

    # Permissions and access
    builder.set("permissionTier", employee_data.get("permissions", "editor"))
    builder.set("environmentAccess", employee_data.get("env_access", ["dev"]))

    # Compliance and licensing
    if compliance_data:
        builder.set("licenseStates", compliance_data.get("licensed_states", []))
        builder.set("certificationLevel", compliance_data.get("certification"))
        builder.set("complianceTraining", compliance_data.get("compliance_complete"))

    # Feature access controls
    builder.set("canPublishGraphs", employee_data.get("can_publish", False))
    builder.set("canModifyTools", employee_data.get("can_modify_tools", False))
    builder.set("canAccessPii", employee_data.get("pii_access", False))

    return builder.build()

print("[OK] build_builder_context() defined")

[OK] build_builder_context() defined


In [24]:
# Test build_builder_context
print("=== Testing build_builder_context ===")
ctx = build_builder_context(
    "emp-12345",
    {
        "role": "claims_adjuster",
        "department": "claims",
        "seniority": "senior",
        "permissions": "publisher",
        "env_access": ["dev", "staging", "prod"],
        "can_publish": True,
        "pii_access": True
    },
    {
        "licensed_states": ["CA", "TX", "NY"],
        "certification": "advanced",
        "compliance_complete": True
    }
)
print(f"[OK] Context key: {ctx.key}")
print(f"     builderRole: {ctx.get('builderRole')}")
print(f"     seniority: {ctx.get('seniority')}")
print(f"     permissionTier: {ctx.get('permissionTier')}")
print(f"     canPublishGraphs: {ctx.get('canPublishGraphs')}")
print(f"     certificationLevel: {ctx.get('certificationLevel')}")

=== Testing build_builder_context ===
[OK] Context key: emp-12345
     builderRole: claims_adjuster
     seniority: senior
     permissionTier: publisher
     canPublishGraphs: True
     certificationLevel: advanced


---
## 12. Session & Execution Context
From: `SKILL.md` lines 715-759

In [25]:
def build_session_context(
    user_id: str,  # Use stable user ID, not session ID as key
    execution_data: dict,
    anonymous: bool = False
) -> Context:
    """
    Build session/execution context.

    IMPORTANT: Use a stable key (user/employee ID), NOT session ID.
    Session IDs should be attributes, not keys, to avoid cardinality explosion.
    """

    # Use stable user ID as key, add session as attribute
    builder = Context.builder(user_id).kind("session")

    # Set anonymous to exclude from dashboard if needed
    if anonymous:
        builder.anonymous(True)

    # Low-cardinality session attributes
    builder.set("invocationReason", execution_data.get("reason"))  # user/scheduled/retry
    builder.set("attemptNumber", min(execution_data.get("attempt", 1), 3))  # cap at 3
    builder.set("isReplay", execution_data.get("is_replay", False))
    builder.set("debugMode", execution_data.get("debug", False))

    # High-cardinality identifiers as private attributes
    if execution_data.get("session_id"):
        builder.set("sessionId", execution_data.get("session_id"))
        builder.private("sessionId")

    if execution_data.get("conversation_id"):
        builder.set("conversationId", execution_data.get("conversation_id"))
        builder.private("conversationId")

    if execution_data.get("request_id"):
        builder.set("requestId", execution_data.get("request_id"))
        builder.private("requestId")

    if execution_data.get("trace_id"):
        builder.set("traceId", execution_data.get("trace_id"))
        builder.private("traceId")

    return builder.build()

print("[OK] build_session_context() defined")

[OK] build_session_context() defined


In [26]:
# Test build_session_context
print("=== Testing build_session_context ===")
ctx = build_session_context(
    "user-stable-123",
    {
        "reason": "user_action",
        "attempt": 1,
        "session_id": "sess-abc-123",
        "conversation_id": "conv-xyz-789",
        "trace_id": "trace-123456"
    }
)
print(f"[OK] Context key: {ctx.key}")
print(f"     invocationReason: {ctx.get('invocationReason')}")
print(f"     attemptNumber: {ctx.get('attemptNumber')}")
print(f"     sessionId (private): {ctx.get('sessionId')}")

=== Testing build_session_context ===
[OK] Context key: user-stable-123
     invocationReason: user_action
     attemptNumber: 1
     sessionId (private): sess-abc-123


---
## 13. Complete Multi-Context Example
From: `SKILL.md` lines 765-838

In [27]:
def build_complete_agent_context(
    user_id: str,
    org_id: str,
    graph_id: str,
    node_id: str,
    mcp_servers: list,
    session_id: str
) -> Context:
    """Build complete multi-context for agent evaluation"""

    multi_builder = Context.multi_builder()

    # 1. User context (insurance employee)
    user_context = Context.builder(user_id).kind("user")
    user_context.set("role", "claims_adjuster")
    user_context.set("department", "claims")
    user_context.set("tier", "professional")
    multi_builder.add(user_context.build())

    # 2. Organization context
    org_context = Context.builder(org_id).kind("organization")
    org_context.set("industry", "insurance")
    org_context.set("complianceTier", "high")
    org_context.set("dataResidency", "us-west")
    multi_builder.add(org_context.build())

    # 3. Agent graph context
    graph_context = Context.builder(graph_id).kind("agentGraph")
    graph_context.set("graphVersion", "2.1")
    graph_context.set("nodeId", node_id)
    graph_context.set("nodeType", "retriever")
    multi_builder.add(graph_context.build())

    # 4. Orchestrator context
    orch_context = Context.builder("orch-prod-west").kind("orchestrator")
    orch_context.set("orchestratorType", "langgraph")
    orch_context.set("executionMode", "async")
    multi_builder.add(orch_context.build())

    # 5. MCP/Toolchain context (use predefined profile)
    toolchain_profile = "claims-standard"  # Predefined profile, not dynamic
    toolchain_context = build_mcp_context(toolchain_profile, mcp_servers, "strict")
    multi_builder.add(toolchain_context)

    # 6. Session context (use user ID as key, session ID as attribute)
    session_context = Context.builder(user_id).kind("session")
    session_context.set("sessionId", session_id)
    session_context.private("sessionId")  # Mark attribute as private
    session_context.set("invocationReason", "user_action")
    multi_builder.add(session_context.build())

    return multi_builder.build()

print("[OK] build_complete_agent_context() defined")

[OK] build_complete_agent_context() defined


In [28]:
# Test build_complete_agent_context
print("=== Testing build_complete_agent_context ===")
ctx = build_complete_agent_context(
    user_id="emp-123",
    org_id="carrier-456",
    graph_id="claims-triage",
    node_id="node-01",
    mcp_servers=[
        {"id": "mcp-claims-db", "vendor": "internal", "riskTier": "medium"},
        {"id": "mcp-docs", "vendor": "internal", "riskTier": "low"}
    ],
    session_id="sess-789"
)
print(f"[OK] Multi-context created")
print(f"     Multiple: {ctx.multiple}")

=== Testing build_complete_agent_context ===
[OK] Multi-context created
     Multiple: True


---
## 14. Context Factory Patterns
From: `SKILL.md` lines 844-905

In [29]:
class ContextFactory:
    """Factory for creating standardized contexts"""

    @staticmethod
    def create_retail_customer(customer_data: dict) -> Context:
        """Create context for retail/e-commerce"""
        builder = Context.builder(customer_data["id"])

        # Customer attributes
        builder.set("customerType", customer_data.get("type", "regular"))
        builder.set("lifetimeValue", customer_data.get("ltv", 0))
        builder.set("purchaseFrequency", customer_data.get("purchase_frequency"))
        builder.set("cartAbandonment", customer_data.get("cart_abandonment_rate"))
        builder.set("preferredCategories", customer_data.get("categories", []))
        builder.set("loyaltyTier", customer_data.get("loyalty_tier", "bronze"))

        # Behavioral attributes
        builder.set("lastPurchaseDays", customer_data.get("days_since_purchase"))
        builder.set("avgOrderValue", customer_data.get("aov"))
        builder.set("returnRate", customer_data.get("return_rate"))

        return builder.build()

    @staticmethod
    def create_saas_user(user_data: dict, account_data: dict) -> Context:
        """Create context for SaaS applications"""
        builder = Context.builder(user_data["id"])

        # User attributes
        builder.set("role", user_data.get("role"))
        builder.set("onboardingCompleted", user_data.get("onboarding_complete"))
        builder.set("lastLogin", user_data.get("last_login"))

        # Account attributes
        builder.set("accountId", account_data.get("id"))
        builder.set("plan", account_data.get("plan"))
        builder.set("mrr", account_data.get("monthly_revenue"))
        builder.set("churnRisk", account_data.get("churn_risk_score"))
        builder.set("healthScore", account_data.get("health_score"))
        builder.set("contractEndDate", account_data.get("contract_end"))

        # Usage attributes
        builder.set("activeUsers", account_data.get("active_user_count"))
        builder.set("storageUsed", account_data.get("storage_gb"))
        builder.set("apiCalls", account_data.get("api_calls_month"))

        return builder.build()

print("[OK] ContextFactory class defined")

[OK] ContextFactory class defined


In [30]:
# Test ContextFactory
print("=== Testing ContextFactory ===")

retail_ctx = ContextFactory.create_retail_customer({
    "id": "customer-123",
    "type": "vip",
    "ltv": 5000,
    "loyalty_tier": "platinum"
})
print(f"[OK] Retail context key: {retail_ctx.key}")
print(f"     customerType: {retail_ctx.get('customerType')}")
print(f"     loyaltyTier: {retail_ctx.get('loyaltyTier')}")

saas_ctx = ContextFactory.create_saas_user(
    {"id": "user-456", "role": "admin"},
    {"id": "account-789", "plan": "enterprise", "monthly_revenue": 10000}
)
print(f"\n[OK] SaaS context key: {saas_ctx.key}")
print(f"     plan: {saas_ctx.get('plan')}")
print(f"     mrr: {saas_ctx.get('mrr')}")

=== Testing ContextFactory ===
[OK] Retail context key: customer-123
     customerType: vip
     loyaltyTier: platinum

[OK] SaaS context key: user-456
     plan: enterprise
     mrr: 10000
